In [5]:
from typing import List, Optional
from pydantic import BaseModel #type: ignore

class Address(BaseModel):
    street: str
    city: str
    postal_code: str

class User(BaseModel): #address field is itself an Address model, not a string/dict. When creating User, Pydantic automatically validates the nested object.
    id: int
    name: str
    address: Address

class Comment(BaseModel):
    id: int
    content: str
    replies: Optional[List['Comment']] = None

# A comment can have replies.
# Each reply is again a Comment.
# This is called a recursive model (self-referential model).
# Why quotes 'Comment'?
# Because the class is not fully defined yet.
# So Pydantic must treat 'Comment' as a forward reference.

Comment.model_rebuild()
# Pydantic must resolve the forward references inside the model.
# convert 'Comment' --> actual Comment class
# validate nested recursive structures
# allow deeply nested comments to work
# Without this, Pydantic would not know what 'Comment' refers to, and you would get an error.

address = Address(
    street = 'street no 8, lane no 5',
    city = "hyderabad",
    postal_code = "504001",
)

user = User(
    id= 1,
    name= 'venu',
    address = address,
)

comment = Comment(
    id=1,
    content="Comment 1",
    replies = [
        Comment(id=2, content="reply 1"),
        Comment(id=3, content="reply 2")
    ]
)

# ---------------- Output Section ----------------

print('\n===== USER DETAILS =====')
print(user.model_dump())  # Convert model to dictionary

print('\n===== MAIN COMMENT =====')
print(comment.model_dump())

print('\n===== PRINT COMMENT TREE =====')

def print_comment_tree(comment, level=0):
    indent = "    " * level
    print(f"{indent}- Comment ID: {comment.id}, Content: {comment.content}")
    if comment.replies:
        for reply in comment.replies:
            print_comment_tree(reply, level+1)

print_comment_tree(comment)


===== USER DETAILS =====
{'id': 1, 'name': 'venu', 'address': {'street': 'street no 8, lane no 5', 'city': 'hyderabad', 'postal_code': '504001'}}

===== MAIN COMMENT =====
{'id': 1, 'content': 'Comment 1', 'replies': [{'id': 2, 'content': 'reply 1', 'replies': None}, {'id': 3, 'content': 'reply 2', 'replies': None}]}

===== PRINT COMMENT TREE =====
- Comment ID: 1, Content: Comment 1
    - Comment ID: 2, Content: reply 1
    - Comment ID: 3, Content: reply 2
